In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/NN_Project/captcha_generation.zip

In [ ]:
#importing libraries
import numpy as np

%matplotlib inline
#to use as command line calls #using inline graphs will come next to code

import matplotlib.pyplot as plt #for graphs
import os #for operating system dependent fucntionality
from keras import layers #for building layers of neural net
from keras.models import Model
from keras.models import load_model
from keras import callbacks #for training logs, saving to disk periodically
import cv2 #OpenCV(Open Source computer vision lib), containg CV algos
import string
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
import tensorflow.keras.backend as K

In [ ]:
#total no of images in dataset
n=len(os.listdir("/content/captcha_generation"))
n

9999

In [ ]:
#defining size of image
imgshape=(50,200,1) #50-height, 200-width, 1-no of channels

In [ ]:
character= string.ascii_uppercase + "0123456789" # All symbols captcha can contain
#nchar = len(character) #total number of char possible
nchar = 62
nchar

62

In [ ]:
#preprocesss image
def preprocess():
  X = np.zeros((n,50,200,1)) #1070*50*200 array with all entries 0
  y = np.zeros((5,n,nchar)) #5*1070*36(5 letters in captcha) with all entries 0

  for i, pic in enumerate(os.listdir("/content/captcha_generation")):
  #i represents index no. of image in directory
  #pic contains the file name of the particular image to be preprocessed at a time

    img = cv2.imread(os.path.join("/content/captcha_generation", pic), cv2.IMREAD_GRAYSCALE) #Read image in grayscale format
    pic_target = pic[:-4]#this drops the .png extension from file name and contains only the captcha for training

    if len(pic_target) < 6: #captcha is not more than 5 letters
      img = img / 255.0 #scales the image between 0 and 1
      img = np.reshape(img, (50, 200, 1)) #reshapes image to width 200 , height 50 ,channel 1

      target=np.zeros((5,nchar)) #creates an array of size 5*36 with all entries 0

      for j, k in enumerate(pic_target):
      #j iterates from 0 to 4(5 letters in captcha)
      #k denotes the letter in captcha which is to be scanned
         index = character.find(k) #index stores the position of letter k of captcha in the character string
         target[j, index] = 1 #replaces 0 with 1 in the target array at the position of the letter in captcha

      X[i] = img #stores all the images
      y[:,i] = target #stores all the info about the letters in captcha of all images

  return X,y

In [ ]:
from tensorflow.keras.layers import Reshape, Permute, LSTM


def create_model_with_lstm():
    img = Input(shape=imgshape)  # Get image as an input of shape (height, width, channels)
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)  # 50*200
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 25*100
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 13*50
    conv3 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)  # To improve the stability of the model

    # Reshape the data to match the expected input shape for the Bidirectional LSTM layers
    permuted = Permute((2, 1, 3))(bn)
    reshaped = Reshape((50, 13 * 64))(permuted)

    # Add Bidirectional LSTM layers here
    blstm1 = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(reshaped)
    blstm2 = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(blstm1)

    # Continue with the rest of your code
    flat = layers.Flatten()(blstm2)

    outs = []
    for _ in range(5):  # For 5 letters of captcha
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)  # Drops 0.5 fraction of nodes
        res = layers.Dense(nchar, activation='sigmoid')(drop)
        outs.append(res)  # Result of layers

    # Compile the model and return it
    model = Model(img, outs)  # Create the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#Create model
model=create_model_with_lstm();
model.summary();

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 200, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 50, 200, 16)          160       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 25, 100, 16)          0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 25, 100, 32)          4640      ['max_pooling2d[0][0]']   

In [ ]:
X,y=preprocess()

In [ ]:
#split the 9999 samples where 8000 samples will be used for training purpose
X_train, y_train = X[:8000], y[:, :8000]
X_test, y_test = X[8000:], y[:, 8000:]

In [ ]:
#Applying the model

hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=100, epochs=60, validation_split=0.2, verbose = 1)
#batch size- 32 defines no. of samples per gradient update
#Validation split=0.2 splits the training set in 80-20% for training nd testing

Epoch 1/60
64/64 [==============================] - 27s 80ms/step - loss: 19.9530 - dense_1_loss: 3.9710 - dense_3_loss: 4.0052 - dense_5_loss: 4.0022 - dense_7_loss: 3.9853 - dense_9_loss: 3.9894 - dense_1_accuracy: 0.0288 - dense_3_accuracy: 0.0266 - dense_5_accuracy: 0.0289 - dense_7_accuracy: 0.0266 - dense_9_accuracy: 0.0295 - val_loss: 19.8593 - val_dense_1_loss: 3.9563 - val_dense_3_loss: 3.9813 - val_dense_5_loss: 3.9721 - val_dense_7_loss: 3.9829 - val_dense_9_loss: 3.9667 - val_dense_1_accuracy: 0.0281 - val_dense_3_accuracy: 0.0181 - val_dense_5_accuracy: 0.0275 - val_dense_7_accuracy: 0.0281 - val_dense_9_accuracy: 0.0294
Epoch 2/60
64/64 [==============================] - 2s 31ms/step - loss: 18.9369 - dense_1_loss: 3.7412 - dense_3_loss: 3.8076 - dense_5_loss: 3.7927 - dense_7_loss: 3.7835 - dense_9_loss: 3.8120 - dense_1_accuracy: 0.0372 - dense_3_accuracy: 0.0344 - dense_5_accuracy: 0.0345 - dense_7_accuracy: 0.0322 - dense_9_accuracy: 0.0297 - val_loss: 19.0955 - val_d

In [ ]:
#graph of loss vs epochs
for label in ["loss"]:
  plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
#Loss on training set
#Finding Loss on training set
preds = model.evaluate(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]])
print ("Loss on training set= " + str(preds[0]))
print ("Accuracy on training set= " + str(preds[10]))

250/250 [==============================] - 5s 19ms/step - loss: 0.1952 - dense_1_loss: 0.0162 - dense_3_loss: 0.0294 - dense_5_loss: 0.0620 - dense_7_loss: 0.0673 - dense_9_loss: 0.0203 - dense_1_accuracy: 0.9958 - dense_3_accuracy: 0.9935 - dense_5_accuracy: 0.9879 - dense_7_accuracy: 0.9879 - dense_9_accuracy: 0.9945
Loss on training set= 0.19522187113761902
Accuracy on training set= 0.9944999814033508


In [ ]:
#Finding loss on test set
preds = model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print ("Loss on testing set= " + str(preds[0]))
print ("Accuracy on testing set= " + str(preds[10]))

63/63 [==============================] - 1s 12ms/step - loss: 0.8699 - dense_1_loss: 0.0790 - dense_3_loss: 0.1343 - dense_5_loss: 0.2681 - dense_7_loss: 0.2981 - dense_9_loss: 0.0905 - dense_1_accuracy: 0.9825 - dense_3_accuracy: 0.9665 - dense_5_accuracy: 0.9515 - dense_7_accuracy: 0.9425 - dense_9_accuracy: 0.9790
Loss on testing set= 0.8699098229408264
Accuracy on testing set= 0.9789894819259644


In [ ]:
#to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    if img is not None: #image foud at file path
        img = img / 255.0 #Scale image
    else:
        print("Not detected");

    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis])) #np.newaxis=1
    #added this bcoz x_train 970*50*200*1
    #returns array of size 1*5*36
    result = np.reshape(res, (5, -1)) #reshape the array
    k_ind = []
    probs = []
    for i in result:
        k_ind.append(np.argmax(i)) #adds the index of the char found in captcha

    capt = '' #string to store predicted captcha
    for k in k_ind:
        capt += character[k] #finds the char corresponding to the index
    return capt

In [ ]:
#Check model on samples
img=cv2.imread('/content/captcha_generation/0BN77.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap=plt.get_cmap('gray'))


In [ ]:
print("Predicted Captcha =",predict('/content/captcha_generation/0BN77.png'))

1/1 [==============================] - 3s 3s/step
Predicted Captcha = DBN77


In [ ]:
#Sample 2
img=cv2.imread('/content/captcha_generation/ZY321.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
print("Predicted Captcha =",predict('/content/captcha_generation/ZY321.png'))

1/1 [==============================] - 0s 37ms/step
Predicted Captcha = ZY3Z1


# Model Summary
<table style="font-size: 30px;">
  <tr>
    <th>Size of the dataset:</th>
    <th>9999</th>
  </tr>
  <tr>
    <th>Number of epochs:</th>
    <th>60</th>
  </tr>
  <tr>
    <th>Batch Size:</th>
    <th>100</th>
  </tr>
  <tr>
    <th>Training Loss</th>
    <th>0.19522</th>
  </tr>
  <tr>
    <th>Training Accuracy</th>
    <th>0.99449</th>
  </tr>
  <tr>
    <th>Testing Loss</th>
    <th>0.86990</th>
  </tr>
  <tr>
    <th>Testing Accuracy</th>
    <th>0.97898</th>
  </tr>
</table>